In [ ]:
import numpy as np
import pickle
#Name: Alvie Thai
#Define Beta Value for Reward/Punishment
beta = 0.1

class State:
    def __init__(self):
  #dash board: 3 x3 array

        self.data = np.zeros((3, 3))
        self.winner = None
        self.hashVal = None
        self.end = None

  # get hash value
    def getHash(self):
        if self.hashVal is None:
            self.hashVal = 0
            for i in self.data.reshape(9):
                if i == -1:
                    i = 2
                self.hashVal = self.hashVal * 3 + i
        return int(self.hashVal)

    # see if there's a winner, or a draw
    def endGame(self):
        if self.end is not None:
            return self.end
        results = []
        # check row
        for i in range(0, 3):
            results.append(np.sum(self.data[i, :]))
        # check columns
        for i in range(0, 3):
            results.append(np.sum(self.data[:, i]))

        # check diagonals
        results.append(0)
        for i in range(0, 3):
            results[-1] += self.data[i, i]
        results.append(0)
        for i in range(0, 3):
            results[-1] += self.data[i, 3 - 1 - i]

        for result in results:
            if result == 3:
                self.winner = 1
                self.end = True
                return self.end
            if result == -3:
                self.winner = -1
                self.end = True
                return self.end

        # whether it's a tie
        sum = np.sum(np.abs(self.data))
        if sum == 3 * 3:
            self.winner = 0
            self.end = True
            return self.end

        # game is still going on
        self.end = False
        return self.end

   #put new symbol to new position
    def nextState(self, i, j, symbol):
        newState = State()
        newState.data = np.copy(self.data)
        newState.data[i, j] = symbol
        return newState

    # print the board
    def show(self):
        for i in range(0, 3):
            print('-------------')
            out = '| '
            for j in range(0, 3):
                if self.data[i, j] == 1:
                    token = 'X'
                if self.data[i, j] == 0:
                    token = '_'
                if self.data[i, j] == -1:
                    token = 'O'
                out += token + ' | '
            print(out)
        print('-------------')

#generate all new state
def AllStatesImpl(currentState, currentSymbol, allStates):
    for i in range(0, 3):
        for j in range(0, 3):
            if currentState.data[i][j] == 0:
                newState = currentState.nextState(i, j, currentSymbol)
                newHash = newState.getHash()
                if newHash not in allStates.keys():
                    isEnd = newState.endGame()
                    allStates[newHash] = (newState, isEnd)
                    if not isEnd:
                        AllStatesImpl(newState, -currentSymbol, allStates)

#Create a dictionary of allStates
def AllStates():
    currentSymbol = 1
    currentState = State()
    allStates = dict()
    allStates[currentState.getHash()] = (currentState, currentState.endGame())
    AllStatesImpl(currentState, currentSymbol, allStates)
    return allStates

# all possible states
allStates = AllStates()

#Human Player
class HumanPlayer:
    def __init__(self, stepSize = beta, exploreRate = 0.1):
        self.symbol = None
        self.currentState = None
        return
    def reset(self):
        return
    def setSymbol(self, symbol):
        self.symbol = symbol
        return
    def addState(self, state):
        self.currentState = state
        return

    #modify STM
    def compensate(self, reward):
        return

    def select(self):
        coordinate_str = input("Enter the row and column in [1,3] (example: 1 3 for 1st row 3rd column): ")
        coordinate_list = coordinate_str.split() # split the input string into a list of strings
        i = int(coordinate_list[0])-1
        j = int(coordinate_list[1])-1
        if self.currentState.data[i, j] != 0:
            print("These coordinates exist. Please input again!")
            return self.select()
        return (i, j, self.symbol)
# AI player
class AIPlayer:
    # step size to update STM
    def __init__(self, stepSize = beta, exploreRate=0.1):
        self.allStates = allStates

        #STM: State Transition Matrix
        self.STM = dict()

        self.stepSize = stepSize
        self.exploreRate = exploreRate
        self.states = []

    def reset(self):
        self.states = []

    def setSymbol(self, symbol):
        self.symbol = symbol
        for hash in self.allStates.keys():
            (state, isEnd) = self.allStates[hash]
            if isEnd:
                if state.winner == self.symbol:
                    self.STM[hash] = 1.0
                else:
                    self.STM[hash] = 0
            else:
                self.STM[hash] = 0.5

    def getSymbol(self):
        return self.symbol

    # accept a state
    def addState(self, state):
        self.states.append(state)

    #modify the probability in STM
    def compensate(self, reward):
        if len(self.states) == 0:
            return
        self.states = [state.getHash() for state in self.states]
        #reward is 1 for reward and 0 for punishment
        target = reward
        for latestState in reversed(self.states):
            if reward == 0:
                #punishment
                value = self.STM[latestState] + self.stepSize * (target - self.STM[latestState])
            else:
                #reward
                value = self.STM[latestState] + self.stepSize * (target - self.STM[latestState])
            self.STM[latestState] = value
            target = value
        self.states = []

    # Select next move
    def select(self):
        state = self.states[-1]
        nextStates = []
        nextPositions = []
        for i in range(3):
            for j in range(3):
                if state.data[i, j] == 0:
                    nextPositions.append([i, j])
                    nextStates.append(state.nextState(i, j, self.symbol).getHash())
        if np.random.binomial(1, self.exploreRate):
            np.random.shuffle(nextPositions)
            self.states = []
            action = nextPositions[0]
            action.append(self.symbol)
            return action

        values = []
        for hash, pos in zip(nextStates, nextPositions):
            values.append((self.STM[hash], pos))
        np.random.shuffle(values)
        values.sort(key=lambda x: x[0], reverse=True)
        action = values[0][1]
        action.append(self.symbol)
        return action

    #save the training data to a file
    def saveData(self):
        fw = open('LOG_[' + str(self.symbol)+']', 'wb')
        pickle.dump(self.STM, fw)
        fw.close()

    # load training data
    def loadData(self):
        fr = open('LOG_[' + str(self.symbol)+']','rb')
        self.STM = pickle.load(fr)
        fr.close()


class TicTacToe:
    # player 1: move first, symbol is 1
    #player 2: move second, symbol is -1
    # @feedback: if True, both players will receive rewards when game is end
    def __init__(self, player1, player2, feedback=True):
        self.p1 = player1
        self.p2 = player2
        self.feedback = feedback
        self.currentPlayer = None
        self.p1Symbol = 1
        self.p2Symbol = -1
        self.p1.setSymbol(self.p1Symbol)
        self.p2.setSymbol(self.p2Symbol)
        self.currentState = State()
        self.allStates = allStates

    # give reward to two players
    def reward(self):
        if self.currentState.winner == self.p1Symbol:
            #First player win
            self.p1.compensate(1)
            self.p2.compensate(0)
        elif self.currentState.winner == self.p2Symbol:
            #Second player win
            self.p1.compensate(0)
            self.p2.compensate(1)
        else:
            #Tie
            self.p1.compensate(0.1)
            self.p2.compensate(0.5)

    def addCurrentState(self):
        self.p1.addState(self.currentState)
        self.p2.addState(self.currentState)

    def reset(self):
        self.p1.reset()
        self.p2.reset()
        self.currentState = State()
        self.currentPlayer = None

    # @show: if True, print each board during the game
    def play(self, show=False):
        self.reset()
        self.addCurrentState()
        while True:
            # set current player
            if self.currentPlayer == self.p1:
                self.currentPlayer = self.p2
            else:
                self.currentPlayer = self.p1
            if show:
                self.currentState.show()
            [i, j, symbol] = self.currentPlayer.select()
            self.currentState = self.currentState.nextState(i, j, symbol)
            hashValue = self.currentState.getHash()
            self.currentState, isEnd = self.allStates[hashValue]
            self.addCurrentState()
            if isEnd:
                if self.feedback:
                    self.reward()
                return self.currentState.winner

def train(trainTimes=1000):
    player1 = AIPlayer()
    player2 = AIPlayer()
    judger = TicTacToe(player1, player2)
    player1Win = 0.0
    player2Win = 0.0
    for i in range(0, trainTimes):
        winner = judger.play(False)
        if winner == 1:
            player1Win += 1
        if winner == -1:
            player2Win += 1
        judger.reset()
    print("Player 1 win rate: ", player1Win / trainTimes)
    print("Player 2 win rate: ",player2Win / trainTimes)
    player1.saveData()
    player2.saveData()

if __name__ == "__main__":
    beta=input("Enter beta value: ")
    train()
    print("")
    print("____GAME____")
    print("The first player's symbol is X, the second player's symbol is O")
    while(True):
      Order = input("Which one will go first (human/AI):")
      player1 = AIPlayer(exploreRate=0)
      player2 = HumanPlayer()
      if Order == "AI":
          judger = TicTacToe(player1, player2)
      elif Order == "human":
          judger = TicTacToe(player2, player1)
      else:
          print("Wrong Input. Please try another time.")
      player1.loadData()
      winner = judger.play(True)
      if winner == player2.symbol:
          judger.currentState.show()
          print("You win!")
      elif winner == player1.symbol:
          judger.currentState.show()
          print("You lose!")
      else:
          judger.currentState.show()
          print("Tie!")
      print("Continue ? (y/n)")
      game_con=input()
      if((game_con=="n") or (game_con=="no")):
        break

